In [1]:
#!pip install lazypredict

In [2]:
__PROJECT_SOURCE__="COLAB"

In [3]:
if __PROJECT_SOURCE__=="COLAB":
    # Import PyDrive and associated libraries.
    # This only needs to be done once per notebook.
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials

    # Authenticate and create the PyDrive client.
    # This only needs to be done once per notebook.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    from google.colab import drive
    drive.mount('/content/drive')
    fname_dir="/content/blood/"
    fname_url="/content/drive/MyDrive/blood/rework/*full.csv"
    fname=fname_url.split("/")[-1]
elif __PROJECT_SOURCE__=="LOCAL":
    fname_dir="sdfsdfsd" #working dir
    fname_url="/content/drive/MyDrive/blood/rework/*agg.csv" #data source dir
    


Mounted at /content/drive


In [4]:
!mkdir $fname_dir

In [5]:
!cp $fname_url $fname_dir

In [6]:
if __PROJECT_SOURCE__=="COLAB":
    drive.flush_and_unmount()
    print('Unmount Google Drive :-(')

Unmount Google Drive :-(


## Feldolgozás

In [7]:
!rm *.hdf5

rm: cannot remove '*.hdf5': No such file or directory


In [8]:
import pandas as pd

In [9]:
fnames_list=["ldl1_full.csv","ldl2_full.csv"]

In [10]:
file_name1=fname_dir+fnames_list[0]
print(file_name1)

/content/blood/ldl1_full.csv


In [11]:
file_name2=fname_dir+fnames_list[1]
print(file_name2)

/content/blood/ldl2_full.csv


In [12]:
df_agg1= pd.read_csv(file_name1)
df_agg1.describe()
df_agg1.drop(df_agg1[df_agg1.absorbance0 < 0].index, inplace=True) # kill the negative elements


In [13]:
df_agg1.head()

,Unnamed: 0,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,...,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,donation_id,temperature,humidity,cholesterol_ldl_value,cholesterol_ldl_human
0,0,0.555104,0.557888,0.560819,0.564807,0.569713,0.571515,0.575181,0.584829,0.593431,0.605127,0.618471,0.629546,0.636551,0.640862,0.641604,0.641446,0.640700,0.638512,0.634634,0.632141,0.628197,0.624323,0.621635,0.618258,0.615647,0.610852,0.607246,0.607107,0.603485,0.601846,0.600708,0.601836,0.602374,0.603096,0.603789,0.606727,0.609770,0.613277,0.617720,...,1.520773,1.510931,1.492422,1.490522,1.483260,1.466496,1.446990,1.446223,1.436076,1.430866,1.415027,1.405551,1.398514,1.384277,1.380456,1.384401,1.365411,1.366780,1.362377,1.347071,1.345513,1.344558,1.342451,1.334845,1.332185,1.332433,1.338564,1.347783,1.326137,1.330770,1.339941,1.359609,1.362426,1.363026,1.342270,10908,36.46,52.76,31.9,0
1,1,0.555762,0.560531,0.558478,0.562081,0.562192,0.570873,0.577154,0.582132,0.593250,0.603150,0.617897,0.628655,0.636400,0.640521,0.640574,0.640428,0.639644,0.637355,0.635599,0.630248,0.627635,0.622413,0.620346,0.616633,0.612760,0.609439,0.606312,0.604100,0.602474,0.601674,0.600377,0.600369,0.601505,0.601641,0.603493,0.605021,0.609614,0.612522,0.616560,...,1.526411,1.502567,1.497602,1.485082,1.481636,1.464493,1.456803,1.441662,1.438423,1.424229,1.419714,1.400824,1.403250,1.397352,1.379809,1.376884,1.367898,1.360034,1.355922,1.349818,1.338001,1.330943,1.335373,1.332000,1.333735,1.335764,1.333988,1.306720,1.347984,1.343536,1.338133,1.331781,1.333470,1.327370,1.339093,10908,36.58,52.64,31.9,0
2,2,0.562597,0.562343,0.568340,0.566400,0.572975,0.578334,0.580465,0.589982,0.598328,0.610434,0.623061,0.633923,0.640617,0.642912,0.644928,0.645464,0.643183,0.641556,0.636517,0.634591,0.630620,0.625786,0.622011,0.619110,0.616875,0.612358,0.608903,0.606762,0.605808,0.603580,0.602655,0.603412,0.602630,0.605225,0.605887,0.607095,0.610909,0.615698,0.617958,...,1.528323,1.507033,1.496102,1.487758,1.477625,1.474076,1.458480,1.446704,1.432638,1.423987,1.406735,1.411351,1.396077,1.393421,1.381700,1.370491,1.374372,1.373860,1.356037,1.354704,1.337940,1.345736,1.332683,1.337675,1.339260,1.342463,1.326266,1.329472,1.346833,1.337076,1.360504,1.335544,1.349121,1.351695,1.353211,10908,36.72,52.54,31.9,0
3,3,0.561160,0.560216,0.561439,0.566591,0.570579,0.573371,0.580483,0.589868,0.598581,0.609787,0.621992,0.632943,0.639635,0.644243,0.645487,0.645152,0.644481,0.640507,0.637796,0.635107,0.630688,0.626547,0.622671,0.619277,0.616344,0.611148,0.608747,0.606468,0.605317,0.604573,0.601598,0.603415,0.602356,0.604682,0.605558,0.607304,0.610898,0.614099,0.618129,...,1.513074,1.508274,1.491448,1.491685,1.475292,1.468794,1.450865,1.449815,1.441932,1.424617,1.425701,1.408045,1.398489,1.389195,1.383651,1.373491,1.374069,1.354400,1.348482,1.343348,1.344841,1.344812,1.338079,1.335832,1.334965,1.331633,1.338109,1.336605,1.328783,1.333058,1.337232,1.319352,1.374021,1.375438,1.340502,10908,36.83,52.48,31.9,0
4,4,0.562610,0.565366,0.564041,0.568373,0.573241,0.577843,0.583221,0.590243,0.601484,0.611506,0.623042,0.635075,0.641987,0.6

In [14]:
df_agg2= pd.read_csv(file_name2)
df_agg2.describe()
df_agg2.drop(df_agg2[df_agg2.absorbance0 < 0].index, inplace=True) # kill the negative elements


In [15]:
df_agg=pd.concat([df_agg1,df_agg2], ignore_index=True)

In [16]:
df_agg.tail()

,Unnamed: 0,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,...,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,donation_id,temperature,humidity,cholesterol_ldl_value,cholesterol_ldl_human
32515,24595,0.506018,0.511588,0.514728,0.522995,0.522961,0.530508,0.537877,0.543076,0.554601,0.568389,0.582809,0.591292,0.599365,0.601744,0.602022,0.603138,0.600151,0.597552,0.594863,0.590940,0.586909,0.583157,0.579254,0.576506,0.570974,0.568377,0.565177,0.562761,0.561213,0.559032,0.557812,0.557912,0.557015,0.557685,0.560666,0.561338,0.563833,0.568147,0.570612,...,1.511238,1.505166,1.490777,1.479463,1.465389,1.454685,1.443616,1.434122,1.426536,1.415761,1.405067,1.397242,1.379932,1.373153,1.364595,1.359029,1.356389,1.352608,1.339140,1.333420,1.324067,1.321493,1.325140,1.333179,1.322949,1.313297,1.329096,1.313718,1.328141,1.324515,1.325210,1.304364,1.303733,1.311444,1.426517,1974,40.75,40.18,130.2,3
32516,24596,0.507788,0.515130,0.514383,0.516559,0.524998,0.530050,0.537561,0.542864,0.555810,0.568044,0.582369,0.592729,0.599622,0.601999,0.602173,0.601870,0.600475,0.598526,0.595201,0.590471,0.586986,0.583303,0.579900,0.576040,0.571166,0.567999,0.565082,0.563578,0.560635,0.559509,0.557579,0.557783,0.556996,0.556796,0.559637,0.561540,0.564576,0.567907,0.571838,...,1.517297,1.504995,1.497658,1.485209,1.468672,1.460962,1.454316,1.433358,1.422211,1.421244,1.412109,1.397289,1.385969,1.381461,1.369660,1.365378,1.355046,1.351141,1.348660,1.334184,1.331807,1.329454,1.328735,1.320480,1.321458,1.331778,1.334629,1.324187,1.316435,1.330567,1.317782,1.328589,1.338975,1.338410,1.306974,1974,40.82,40.13,130.2,3
32517,24597,0.507435,0.509331,0.511466,0.520766,0.523352,0.528844,0.534432,0.541395,0.553505,0.566188,0.580461,0.591363,0.598073,0.598689,0.601007,0.600427,0.599137,0.595049,0.592260,0.589254,0.584750,0.581037,0.577817,0.574073,0.570079,0.566629,0.563921,0.562093,0.559176,0.558484,0.557063,0.555998,0.556070,0.556430,0.558351,0.559575,0.563627,0.566719,0.569755,...,1.520466,1.506067,1.488761,1.481726,1.469406,1.460918,1.450390,1.440174,1.426757,1.421634,1.404178,1.401206,1.391210,1.384144,1.370918,1.362318,1.356493,1.354322,1.332693,1.332460,1.331159,1.326356,1.321987,1.332561,1.326842,1.329386,1.321799,1.317739,1.322352,1.335085,1.335843,1.302808,1.314103,1.333720,1.365253,1974,40.87,40.08,130.2,3
32518,24598,0.504340,0.512301,0.514911,0.517578,0.525679,0.529063,0.535851,0.540529,0.551863,0.566400,0.580162,0.590489,0.596986,0.598853,0.600226,0.599622,0.597620,0.596002,0.591475,0.588685,0.584692,0.581810,0.576919,0.572615,0.570243,0.566556,0.563359,0.561002,0.560049,0.557929,0.556316,0.556340,0.555732,0.556763,0.557205,0.559490,0.562675,0.566376,0.570217,...,1.517641,1.507594,1.493915,1.484648,1.471439,1.460701,1.456381,1.432987,1.420348,1.413911,1.410280,1.389240,1.387780,1.384702,1.375359,1.362545,1.351091,1.346058,1.352913,1.341091,1.325575,1.331336,1.336737,1.325752,1.333823,1.327902,1.333474,1.325987,1.323078,1.320766,1.305506,1.317392,1.308363,1.303539,1.309175,1974,40.94,40.02,130.2,3
32519,24599,0.505921,0.509828,0.512678,0.519219,0.523278,0.531142,0.535303,0.542502,0.553941

In [17]:
df_agg.describe()

,Unnamed: 0,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,...,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,donation_id,temperature,humidity,cholesterol_ldl_value,cholesterol_ldl_human
count,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,...,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000,32520.000000
mean,10247.987085,0.520451,0.522427,0.525972,0.530680,0.536854,0.543551,0.549648,0.555232,0.563119,0.574407,0.587313,0.598591,0.605160,0.608182,0.609431,0.609150,0.607543,0.605090,0.602129,0.598775,0.595773,0.592707,0.589487,0.585683,0.582415,0.579220,0.576781,0.574689,0.572943,0.571480,0.570644,0.570247,0.570480,0.571400,0.572933,0.575169,0.577869,0.581283,0.585061,...,1.497814,1.485553,1.474109,1.463112,1.450626,1.439371,1.428135,1.417230,1.406798,1.398556,1.385953,1.377019,1.368533,1.360171,1.351819,1.344645,1.337181,1.330049,1.323455,1.316835,1.311906,1.307739,1.304195,1.301982,1.301178,1.300802,1.300394,1.299197,1.295539,1.289453,1.282270,1.275940,1.272953,1.287608,1.275556,6927.533210,42.111418,39.222955,111.436592,1.485240
std,7217.494367,0.041112,0.040721,0.040812,0.040743,0.040851,0.041212,0.041108,0.041028,0.040799,0.040583,0.040467,0.040440,0.040371,0.040357,0.040423,0.040384,0.040286,0.040235,0.040161,0.040257,0.040208,0.040203,0.040219,0.040330,0.040482,0.040505,0.040565,0.040594,0.040617,0.040604,0.040585,0.040536,0.040548,0.040518,0.040482,0.040474,0.040484,0.040568,0.040562,...,0.092871,0.091442,0.090024,0.088669,0.087008,0.085677,0.084387,0.083439,0.082445,0.081613,0.080247,0.079475,0.078764,0.078134,0.077412,0.076854,0.076236,0.075665,0.075127,0.074675,0.074371,0.074165,0.074055,0.074165,0.074513,0.075227,0.076246,0.077813,0.080530,0.084704,0.090256,0.096075,0.100996,0.108040,0.107367,3092.146553,3.516987,8.996599,44.266598,1.030351
min,0.000000,0.392912,0.394812,0.400689,0.403960,0.408725,0.413678,0.419136,0.426281,0.436464,0.448900,0.461060,0.472540,0.479179,0.481304,0.483719,0.483216,0.481971,0.480212,0.477067,0.473347,0.470453,0.467312,0.464336,0.460893,0.457083,0.453718,0.452251,0.450437,0.448239,0.447033,0.446642,0.446144,0.446081,0.447483,0.448803,0.451550,0.454424,0.457708,0.461049,...,1.183231,1.176505,1.165242,

In [18]:

import tensorflow as tf
from keras.layers import InputLayer, Dense, LSTM, Input, Dropout,Embedding, Flatten,LayerNormalization
from keras.models import Sequential, Model

import keras.optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.backend import clear_session
from tensorflow.keras.losses import mean_absolute_percentage_error, huber,kld,mse

from keras.preprocessing.sequence import pad_sequences
import numpy as np


In [19]:
X_columns=list(df_agg.columns[1:-5])
X_columns.append("temperature")
y_columns=df_agg.columns[-1] # the category is at the end of columns
X_=df_agg[X_columns]
y_=df_agg[y_columns]

In [20]:
y_.replace(3,2, inplace=True) # the generated file contains  0,1,3 as the category label :-( 

In [21]:
#y_.replace(2,0, inplace=True)

In [22]:
y_=np.array(y_)

In [23]:
y_=y_.reshape(-1,1)

In [24]:
y_columns

'cholesterol_ldl_human'

In [25]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

X_columns=df_agg.columns[1:-5]
y_columns=df_agg.columns[-1] # the category is at the end of columns
X_=df_agg[X_columns]
y_=df_agg[y_columns]

y_.replace(3,2, inplace=True) # the generated file contains  0,1,3 as the category label :-( 

y_=np.array(y_)
y_=y_.reshape(-1,1)



__X_SCALER__="normal"

__DNN_MODE__="regression"

__Y_SCALER__="minmax"

if __DNN_MODE__=="regression":
    y_columns=df_agg.columns[-2] # the category is at the  -2  column
    y_=df_agg[y_columns]
    y_=np.array(y_)
    y_=y_.reshape(-1,1)


elif __DNN_MODE__=="classification":
    y_columns=df_agg.columns[-1] # the category is at the end of columns
    y_=df_agg[y_columns]
    y_.replace(3,2, inplace=True) # the generated file contains  0,1,3 as the category label :-( 
    y_=np.array(y_)
    y_=y_.reshape(-1,1)

y=y_
print(y[0:10])

if __X_SCALER__=="normal":
    X_normalizer = StandardScaler() 
    X=X_normalizer.fit_transform(X_)

if __Y_SCALER__=="normal":
    y_normalizer = StandardScaler() 
    y=y_normalizer.fit_transform(y_,)
    print(f"Itt: {y}")

if __X_SCALER__=="minmax":
    X_minmax = MinMaxScaler() 
    X=X_minmax.fit_transform(X_)

if __Y_SCALER__=="minmax":
    y_minmax = MinMaxScaler() 
    y=y_minmax.fit_transform(y_)
    




[[31.9]
 [31.9]
 [31.9]
 [31.9]
 [31.9]
 [31.9]
 [31.9]
 [31.9]
 [31.9]
 [31.9]]


In [26]:
'''
import wandb
from wandb.keras import WandbCallback

wandb.init(config={"hyper": "parameter"})
'''



'\nimport wandb\nfrom wandb.keras import WandbCallback\n\nwandb.init(config={"hyper": "parameter"})\n'

In [27]:
def blood_model_2():
    #clear_session()

    keras.backend.clear_session()

    input_len=170

    print(input_len)
    output_size=24
    drop_frac0=0.0 
    drop_frac1=0.0



    input1=Input(shape=(input_len,))

    #flatt=Flatten()(lstm1)

    non=42
    #initializer = tf.keras.initializers.LecunNormal()
    #initializer=tf.keras.initializers.LecunUniform()
    #initializer=tf.keras.initializers.HeUniform(    seed=None)
    #initializer= tf.keras.initializers.RandomNormal(    mean=3.0, stddev=0.05, seed=None)

    initializer="HeNormal"
    d1=Dense(1000,activation="sigmoid",kernel_initializer=initializer)(input1)
    d1=Dense(100,activation="sigmoid",kernel_initializer=initializer)(d1)
    #d1=Dense(100,activation="selu",kernel_initializer=initializer)(d1)


    #softmax
    pred=Dense(1,activation="sigmoid",)(d1)

    
    
    model = Model(inputs=input1, outputs=pred)

    opt = tf.keras.optimizers.Adamax(learning_rate=0.001)


    lossfn = tf.keras.metrics.SparseCategoricalCrossentropy()

    model.compile(loss="CategoricalCrossentropy",
        optimizer=opt,
        metrics=["Accuracy"])
    return(model)

In [28]:
!pip install wandb
!wandb login

     |████████████████████████████████| 1.7 MB 12.7 MB/s 
     |████████████████████████████████| 97 kB 6.1 MB/s 
     |████████████████████████████████| 140 kB 55.7 MB/s 
     |████████████████████████████████| 180 kB 65.8 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=2aa015b19653db401c9e26f658deb938a79250bd293a8da9ae08c8c86c7c3dff
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=6ac82aa552775513bd5d9686c4ff2647cf38e63d0560630793118ee9ae1d5416
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or 

In [29]:
import wandb
from wandb.keras import WandbCallback

wandb.init(project="BM_Norm_5", entity="sipoczlaszlo")

wandb: Currently logged in as: sipoczlaszlo (use `wandb login --relogin` to force relogin)


In [30]:
def blood_model_regression():
    #clear_session()

    keras.backend.clear_session()

    input_len=170

    print(input_len)
    output_size=24
    drop_frac0=0.00000 
    drop_frac1=0.0



    input1=Input(shape=(input_len,))

    #flatt=Flatten()(lstm1)

    non=42
    #initializer = tf.keras.initializers.LecunNormal()
    #initializer=tf.keras.initializers.LecunUniform()
    #initializer=tf.keras.initializers.HeUniform(    seed=None)
    #initializer= tf.keras.initializers.RandomNormal(    mean=3.0, stddev=0.05, seed=None)
    # 5512 relu  256 relu  
    initializer="HeNormal"
    initializer2="GlorotNormal"
    initializer = tf.keras.initializers.HeNormal()
    d1=Dense(12512,activation="relu",kernel_initializer=initializer)(input1)
    d1=Dropout(drop_frac0)(d1)
    
    d1=Dense(2156,activation="relu",kernel_initializer=initializer)(d1)
    d1=Dense(216,activation="relu",kernel_initializer=initializer)(d1)
    
    #d1=Dropout(drop_frac1)(d1)
    #d1=Dense(128,activation="tanh",kernel_initializer=initializer2)(d1)
    # d1=Dropout(drop_frac1)(d1) 

    #softmax
    pred=Dense(1, kernel_initializer=initializer)(d1)

    
    
    model = Model(inputs=input1, outputs=pred)

    opt = tf.keras.optimizers.SGD(learning_rate=0.0000001)


    lossfn = tf.keras.metrics.SparseCategoricalCrossentropy()

    model.compile(loss="MSE",
        optimizer=opt,
        metrics=["MSE"])
    return(model)

In [31]:
__DNN_MODE__

'regression'

In [32]:
if __DNN_MODE__=="classification":
    model_name="classification_"
    def scheduler(epoch, lr):
        return 0.001

    callback_LR = tf.keras.callbacks.LearningRateScheduler(scheduler)

    callbacks = [callback_LR,
            
            #savemodela,
            ModelCheckpoint(filepath=model_name+"_{val_loss:.5f}_{loss:.5f}_.hdf5", monitor='val_loss',
                            verbose=1, save_best_only=True, mode='min')]



In [33]:
if __DNN_MODE__=="regression":
    model_name="regression_"
    def scheduler(epoch, lr):
       #return 0.00003
        
        maxx=0.000003
        minn=0.0001
        frekvency=50
        o=(epoch % frekvency)/frekvency * (maxx-minn)+minn
        return o

        
        if epoch<50:
            return 0.001
        elif epoch <100:
            return 0.001
        elif epoch <1200:
            return 0.0001
        return 0.001    


    callback_LR = tf.keras.callbacks.LearningRateScheduler(scheduler)

    callbacks = [callback_LR,
            
            #savemodela,
            ModelCheckpoint(filepath=model_name+"_{loss:.5f}_{val_loss:.5f}_.hdf5", monitor='val_loss',
                            verbose=1, save_best_only=True, mode='min')]



In [34]:
if __DNN_MODE__=="classification":
    new_model_2=blood_model_2()

In [35]:
if __DNN_MODE__=="regression":
    new_model_2=blood_model_regression()

170


In [36]:
!rm *.hdf5

rm: cannot remove '*.hdf5': No such file or directory


In [37]:
X

array([[ 0.84291107,  0.87084623,  0.85384503, ...,  0.88592384,
         0.69807174,  0.62136917],
       [ 0.85890338,  0.93575649,  0.79650173, ...,  0.59921558,
         0.36804195,  0.59178345],
       [ 1.02517371,  0.98026626,  1.03813112, ...,  0.75418405,
         0.59318911,  0.72327235],
       ...,
       [-0.3165878 , -0.32159651, -0.35542714, ...,  0.4074519 ,
         0.42681638,  0.83543855],
       [-0.39189285, -0.2486709 , -0.27101785, ...,  0.35061772,
         0.14746353,  0.31313052],
       [-0.35342492, -0.3093902 , -0.3257258 , ...,  0.91233697,
         0.57188824,  0.7372212 ]])

In [38]:
 from sklearn.model_selection import train_test_split
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)


In [ ]:
for i in range(5):
    if __DNN_MODE__=="regression":
        new_model_2.summary()
        history=new_model_2.fit(X_train,
          y_train,validation_data=(X_test,y_test),
          epochs=10, 
          batch_size=3,
          validation_split=0.1,
          verbose=1,
          callbacks=[callbacks,WandbCallback()],
          shuffle=True
          
          )


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 170)]             0         
                                                                 
 dense (Dense)               (None, 12512)             2139552   
                                                                 
 dropout (Dropout)           (None, 12512)             0         
                                                                 
 dense_1 (Dense)             (None, 2156)              26978028  
                                                                 
 dense_2 (Dense)             (None, 216)               465912    
                                                                 
 dense_3 (Dense)             (None, 1)                 217       
                                                                 
Total params: 29,583,709
Trainable params: 29,583,709
Non-tra

In [ ]:
---

In [ ]:
#new_model_2.load_weights("./_col_0.00098_0.00173_.xhdf5")

In [ ]:
new_model_2.load_weights("./regression__0.00420_0.00446_.hdf5")

In [ ]:
y_pred=new_model_2.predict(X)
result_df=pd.DataFrame()
y_pred_col=[i[0] for i in y_pred]
y_col=[i[0] for i in y]

result_df["y"]=y_col
result_df["y_pred"]=y_pred_col

In [ ]:
import plotly.express as px

fig = px.scatter(result_df, x="y", y="y_pred",width=600, height=400 )
fig.show()

In [ ]:
---

In [ ]:


from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score,mean_absolute_error


clf = RandomForestRegressor(max_depth=70, random_state=0,n_estimators=300)
clf.fit(X_train, y_train)

randomforest_predict=clf.predict(X_train)


In [ ]:
print(f"Accuracy: {mean_absolute_error(y_train,randomforest_predict)}")

In [ ]:
randomforest_predict=clf.predict(X_test)
print(f"Accuracy: {mean_absolute_error(y_test,randomforest_predict)}")

In [ ]:
---

In [ ]:
y

In [ ]:
clf = LazyClassifier(verbose=1,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X[:-100], X[-100:], y[:-100], y[-100:])

In [ ]:
print(models)

In [ ]:
yo=[i[0] for i in y_]
yo

In [ ]:
X

In [ ]:
from lazypredict.Supervised import LazyRegressor
reg = LazyRegressor(verbose=2, ignore_warnings=False, custom_metric=None)
models,predictions = clf.fit(X[:-100], X[-100:], y[:-100], y[-100:])

In [ ]:
print(models)

In [ ]:
x_counter=1

In [ ]:
X_train=X[:-1*x_counter]
X_test=X[-1*x_counter:]
y_train=yo[:-1*x_counter]
y_test =yo[-1*x_counter:]
